In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import seaborn as sns
import FinancialMachineLearning as fml

### Chapter 9. Hyper-Parameter Tuning with Cross-Validation
#### Exercise 1
8장의 `getTestData` 함수를 사용해 10개의 특성을 가진 10,000개의 관측값을 생성하라. 이 중 5개는 정보성, 5개는 잡음이다

In [2]:
X, y = fml.getTestData(n_features = 10, n_informative = 5, n_redundant = 0, n_samples = 3000)

**(a)** `GridSearchCV`를 10겹 CV를 사용해 RBF 커널을 가지고 있는 SVC의 최적 Hyper Parameter C, gamma를 찾아라, 여기서 param_grid = {'C' : [1E-2, 1E-1, 1, 10, 100], 'gamma' : [1E-2, 1E-1, 1, 10, 100]}이고, scoring은 neg_log_loss이다.

In [3]:
from sklearn.svm import SVC

clf = SVC(kernel = 'rbf', probability = True)
param_grid = {'C' : [0.01, 0.1, 1, 10, 100], 'gamma' : [0.01, 0.1, 1, 10, 100]}

In [4]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import time
start = time.time()
gridSearch = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = 'neg_log_loss', n_jobs = -1, cv = 10)
gridSearch.fit(X, y['bin'], sample_weight = y['w'])
end = time.time()

print('Calculating time : %.5f second' %(end - start))

Calculating time : 116.94082 second


**(b)** Grid에 노드는 몇 개 있는가?

In [5]:
from sklearn.model_selection import ParameterGrid
grid = ParameterGrid(param_grid)
num = 0
for i in grid : num += 1

print(num)

25


**(c)** 최적해를 찾으려면 몇 번의 적합화가 필요한가?

25번의 적합화 과정이 필요하다

**(d)** 이 해법을 찾는 데 걸린 시간은 얼마인가?

- Calculating time : 6305.15831 second

Google Colab Pro기준, 6305초가 걸렸다. 즉, 1시간 5분 정도가 걸렸다

**(e)** 최적 결과는 어떻게 얻을 수 있는가?

In [6]:
gridSearch.best_params_

{'C': 100, 'gamma': 0.1}

**(f)** 최적 Parameter 조합의 교차 검증 점수는 얼마인가?

In [7]:
gridSearch.best_score_

-0.26197533367302467

**(g)** SVC에 Sample Weights를 전달하는 방법은 무엇인가?

적합화 과정에서 sample weight의 pandas.Series형태의 column을 추가해주면 된다

#### Exercise 2
`RandomizedSearchCV`를 10겹 Cross Validation을 사용해 RBF 커널을 갖고 있는 SVC의 최적 Hyper Parameter `C`, `gamma`를 찾아라. 여기서 `param_distributions = {'C' : logUniform(a = 1e-2, b = 1e2), 'gamma' : logUniform(a = 1e-2, b = 1e2)}, n_iter = 25`이고, scoring function은 `neg_log_loss`다

In [8]:
clf = SVC(kernel = 'rbf', probability = True)
param_distributions = {'C' : fml.logUniform(a = 1e-2, b = 1e2), 'gamma': fml.logUniform(a = 1e-2, b = 1e2)}

In [9]:
start = time.time()
gs_rand = RandomizedSearchCV(estimator = clf, param_distributions = param_distributions, n_iter = 25, scoring = 'neg_log_loss', n_jobs = -1, cv = 10)
gs_rand.fit(X, y['bin'], sample_weight = y['w'])
end = time.time()

print('Calculating time : %.5f second' %(end - start))

Calculating time : 124.84532 second


**(b)** 이 해법을 찾는 데 걸린 시간은 얼마인가?

- Calculating time : 6203.48227 second

In [10]:
print('Calculating time : %.5f second' %(end - start))

Calculating time : 124.84532 second


**(c)** 최적의 parameter 조합이 Exercise 1에서 찾은 것과 유사한가?

In [11]:
gs_rand.best_params_

{'C': 85.28066300624138, 'gamma': 0.16256126238662402}

**(d)** 최적 parameter 조합의 Cross Validation score는? Exercise 1의 Cross Validation 점수와 비교하면 어떠한가?

In [12]:
gs_rand.best_score_

-0.28739751605596847

Exercise 1에 비해서 점수가 더 높게 나왔다

#### Exercise 3
Exercise 1으로부터

**(a)** 1의 a로부터 샘플 내 예측 결과의 Sharpe Ratio를 계산하라

In [13]:
def get_IS_sharpe_ratio(clf) -> float:
    best_estimator_ind = np.argmin(clf.cv_results_['rank_test_score'])
    mean_score = clf.cv_results_['mean_test_score'][best_estimator_ind]
    std_score = clf.cv_results_['std_test_score'][best_estimator_ind]
    if mean_score < 0:
        return -mean_score / std_score
    else:
        return mean_score / std_score

In [14]:
print(f'Sharpe ratio: {get_IS_sharpe_ratio(gridSearch)}')

Sharpe ratio: 1.3672823120162692


**(b)** 1의 a를 반복하라. 이번에는 scoring function으로 `accuracy`를 사용하라. Tuning된 Hyper Parameter로부터 도출된 샘플 내 예측을 계산해 보라

In [15]:
clf = SVC(kernel = 'rbf', probability = True)
param_grid = {'C' : [0.01, 0.1, 1, 10, 100], 'gamma' : [0.01, 0.1, 1, 10, 100]}

start = time.time()
gridSearch_acc = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = 'accuracy', n_jobs = -1, cv = 10)
gridSearch_acc.fit(X, y['bin'], sample_weight = y['w'])
end = time.time()

print('Calculating time : %.5f second' %(end - start))

Calculating time : 114.44554 second


In [18]:
print(f'Sharpe ratio: {get_IS_sharpe_ratio(gridSearch_acc)}')

Sharpe ratio: 10.400804836898782


**(c)** 어느 Scoring Function을 사용하였을 때 더 높은 Sharpe Ratio가 나타나는가?

Accuracy를 사용하였을 때 월등히 높게 나온다

#### Exercise 4
Exercise 2로부터

**(a)** 2의 a로부터 샘플 내 예측의 Sharpe Ratio를 계산하라

In [16]:
print(f'Sharpe ratio: {get_IS_sharpe_ratio(gs_rand)}')

Sharpe ratio: 1.4450359056549207


**(b)** 2의 a를 반복하라. 이번에는 Scoring Function으로 `accuracy`를 사용하라. Tuning된 Hyper Parameter로부터 도출된 샘플 내 에측을 계산해 보라

In [17]:
clf = SVC(kernel = 'rbf', probability = True)
param_distributions = {'C' : fml.logUniform(a = 1e-2, b = 1e2), 'gamma': fml.logUniform(a = 1e-2, b = 1e2)}

start = time.time()
gs_rand_acc = RandomizedSearchCV(estimator = clf, param_distributions = param_distributions, n_iter = 25, scoring = 'accuracy', n_jobs = -1, cv = 10)
gs_rand_acc.fit(X, y['bin'], sample_weight = y['w'])
end = time.time()

print('Calculating time : %.5f second' %(end - start))

Calculating time : 119.40575 second


In [19]:
print(f'Sharpe ratio: {get_IS_sharpe_ratio(gs_rand_acc)}')

Sharpe ratio: 9.870073646705027


**(c)** 어느 Scoring Function을 사용하였을 때 더 높은 Sharpe Ratio가 나타나는가?

Accuracy를 사용하였을 때 월등히 높게 나타난다

#### Exercise 5
log loss function, $L[Y,P]$의 정의를 읽어 보라

**(a)** `neg_log_loss`가 음수 로그 손실인 $-L[Y,P]$로 정의된 이유는 무엇인가?

정확도는 높은 확률로 잘못된 매수 예측을 한 경우와 낮은 확률로 잘못 매수 예측을 한 경우를 동일하게 취급한다. 게다가 정확도는 높은 확률의 실수와 낮은 확률의 성공을 서로 상쇄시켜 버릴 수 있다. `neg_log_loss`를 사용하면 분류된 확률을 확인할 수 있다. 음수 로그를 쓰는 이유는 직관적으로 보기 좋게 하기 위함이 가장 큰 이유이다


**(b)** 음수 로그 손실이 아니라 로그 손실을 최대화하면 어떤 결과가 도출되는가?

로그 손실을 최대화하도록 설정하면 잘못된 방향으로 학습이 될 것이다. 손실함수가 커지는 지점을 목적으로 두기 때문이다

#### Exercise 6
예측의 신뢰도와 무관하게 동일한 크기로 베팅하는 투자 전략을 고려해 보자. 이 경우 Hyper Parameter Tuning을 위해서는 Accuracy와 Cross Entropy 손실 중 어느 것이 Scoring Function으로서 더 적절한가?